## 데이터 준비

- 샘플 데이터를 받아서 fine-tuning 가능한 형태로 구성합니다.



### 샘플 데이터셋 활용

- 링크 : https://huggingface.co/datasets/sean0042/KorMedMCQA
- train, dev, test 로 이뤄져 있음.


### Format 예시

- Solar 예시: https://huggingface.co/upstage/SOLAR-10.7B-Instruct-v1.0
- IFT 시 사용 format
```
<s> ### User:
Hello?

### Assistant:
Hello, how can I assist you today? Please feel free to ask any questions or request help with a specific task.</s>
```

In [ ]:
from datasets import load_dataset

In [ ]:

data_path = "sean0042/KorMedMCQA"

doctor = load_dataset(path=data_path, name="doctor")
nurse = load_dataset(path=data_path,name="nurse")
pharmacist = load_dataset(path=data_path, name="pharm")

In [ ]:
import pandas as pd

In [ ]:
def merge_data(data_type):
    doctor_set = pd.DataFrame(doctor[data_type])
    nurse_set = pd.DataFrame(nurse[data_type])
    pharmacist_set = pd.DataFrame(pharmacist[data_type])
    
    return pd.concat([doctor_set, nurse_set, pharmacist_set])

In [ ]:
train_df = merge_data('train')
val_df = merge_data('dev')
test_df = merge_data('test')

In [ ]:
train_df

### 데이터 준비

- 데이터를 적절하게 전처리하여 원하는 형태로 만들어 놓도록 합니다.

In [ ]:
def refine_text(df):
    answer_list = []
    for row in df.itertuples():
        question = row[5]
        answer_idx = int(row[11])
        answer = row[5 + answer_idx]
        answer_list.append(answer)
    
    input_df = df["question"].to_frame()
    final_df = input_df.assign(output=answer_list)
    # final_df = df.assign(output=answer_list)
    return final_df


def refine_text_v2(df):
    answer_list = []
    for row in df.itertuples():
        question = row[5]
        answer_idx = int(row[11])
        # answer = row[5 + answer_idx]
        
        answer = f"질문에 대한 후보는 {row[6]}, {row[7]}, {row[8]}, {row[9]}, {row[10]} 가 될 수 있는데, 가장 올바른 정답은 {row[5 + answer_idx]} 입니다."
        answer_list.append(answer)
    
    input_df = df["question"].to_frame()
    final_df = input_df.assign(output=answer_list)
    # final_df = df.assign(output=answer_list)
    return final_df


In [ ]:
train_set = refine_text_v2(train_df)
val_set = refine_text_v2(val_df)
# test_set = refine_text_v2(test_df)

In [ ]:
train_set


In [ ]:
train_set['question_length'] = train_set['question'].apply(len)
train_set['output_length'] = train_set['output'].apply(len)
print(train_set.describe())
train_set

In [ ]:
cutoff_len = 1024

In [ ]:
print(f"Sample question: {train_set.iloc[0].question}")
print(f"Sample answer: {train_set.iloc[0].output}")

### Tokenized 된 데이터 준비

- 실제 학습은 일반적인 text가 아니라 token 값을 활용하게 되므로, text를 token으로 변환해 놓도록 합니다.
- 학습을 위한 프롬프트 형태로 만들어서 instruction tuning을 합니다. 위의 포맷을 참고해 주세요. 해당 포맷을 반드시 사용할 필요는 없지만, fine-tuning을 하기 위한 모델의 포맷을 따라가는 것이 좋습니다.
- 여기서 실제 학습에 활용되는 값은 `input_ids` 이며, 추가적으로 `labels`나 `attention_mask`가 활용됩니다.

In [ ]:
%store -r

In [ ]:
model_download_path

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_download_path)

In [ ]:
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

In [ ]:
def prepare_dataset(dataset):
    dataset = dataset.sample(frac=1).reset_index(drop=True)
    
    labels = []
    inputs = []
    masks = []
    for row in dataset.itertuples():
        question = row[1]
        answer = row[2]
#         prompt = f"""<s> ### User:
# {question}

# ### Assistant:
# {answer} </s>
# """
        
        prompt = f"""### User:
{question}

### Assistant:
{answer}
"""
        
        tokenized_result = tokenize(prompt)
        inputs.append(tokenized_result['input_ids'])
        labels.append(tokenized_result['labels'])
        masks.append(tokenized_result['attention_mask'])
    
    dataset = dataset.assign(input_ids=inputs)
    dataset = dataset.assign(labels=labels)
    dataset = dataset.assign(attention_mask=masks)
    
    return dataset
        
        

In [ ]:
train_dataset = prepare_dataset(train_set)
train_dataset

In [ ]:
val_dataset = prepare_dataset(val_set)

### 데이터 저장

- 데이터를 로컬에 저장해서 로컬 테스트 용도로 활용합니다.
- 필요하다면 s3에 저장해 놓을 수 있습니다.

In [ ]:
from datasets import Dataset

hf_train = Dataset.from_pandas(train_dataset)
hf_val = Dataset.from_pandas(val_dataset)

In [ ]:
import os

hf_train.save_to_disk(os.path.join("dataset", "train"))
hf_val.save_to_disk(os.path.join("dataset", "val"))